In [1]:
!python -m pip install obsws-python
!python -m pip install keyboard
!python -m pip install -U obsws-python

In [2]:
import time
from threading import Thread, Barrier
import obsws_python as obs

## Added imports for event logging via keyboard buttons and csv outputs
import csv
from datetime import datetime, timezone
from pathlib import Path
import keyboard
import threading

TARGETS = [ 
    # IP address, the websocket port, obs websocket password, name for printing messages
    {"host": "10.0.0.137", "port": 4455, "password": "GARN_air", "name": "AIR COMPUTER"},
    ## {"host": "192.168.0.141", "port": 4455, "password": "GARN_air", "name": "AIR COMPUTER"},
    {"host": "10.0.0.52", "port": 4455, "password": "GARN_ground", "name": "GROUND COMPUTER"},
    ## {"host": "192.168.0.142", "port": 4455, "password": "GARN_ground", "name": "GROUND COMPUTER"},
]

## Added a mapping for keyboard buttons to event names
EVENT_KEYS = {
    "1": "Stimulus onset",
    "2": "Condition start",
    "3": "Condition end",
    "space": "Default manual marker",
}

## Added an exit key to end the logger
EXIT_KEYS = {"esc"}

## Name for the CSV file
LOG_PATH = Path("mdl_obs_events.csv")

def start_record_job(tgt, go_barrier):
    # Requests client connection to OBS
    cl = obs.ReqClient(host=tgt["host"], port=tgt["port"], password=tgt["password"], timeout=3)
    
    # Asks OBS whether recording is already active
    try:
        status = cl.get_record_status()
        if getattr(status, "output_active", False):
            print(f'[{tgt["name"]}] Already recording.')
            return
        
        # The following ensures all OBS recordings start simultaneously
        go_barrier.wait()
        cl.start_record()
        
        # Wait for OBS to let OBS update its state
        time.sleep(0.2)
        
        # Asks again for status
        confirm = cl.get_record_status()
        if getattr(confirm, "output_active", False):
            print(f'[{tgt["name"]}] Recording started.')
        else:
            print(f'[{tgt["name"]}] Failed to start recording.')
            
    # error prints if anything goes wrong
    except Exception as e:
        print(f'[{tgt["name"]}] ERROR: {e}')
        
def stop_record_job(tgt):
    """Stop recording on one OBS host, with a brief poll to confirm."""
    cl = obs.ReqClient(host=tgt["host"], port=tgt["port"],
                       password=tgt["password"], timeout=3)
    name = tgt.get("name", f'{tgt["host"]}:{tgt["port"]}')
    try:
        status = cl.get_record_status()
        if not getattr(status, "output_active", False):
            print(f'[{name}] Not recording.')
            return

        cl.stop_record()

        # Poll up to ~5s until output_active becomes False
        for _ in range(50):
            time.sleep(0.1)
            confirm = cl.get_record_status()
            if not getattr(confirm, "output_active", False):
                print(f'[{name}] Recording stopped.')
                return

        print(f'[{name}] Stop requested; recorder still reporting active after timeout.')
    except Exception as e:
        print(f'[{name}] ERROR: {e}')

def init_csv_new(path: Path):
    """Always overwrite the CSV file with a fresh header row."""
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["Event Name", "Clock Time (UTC ISO)", "Elapsed Time (s)"])

# def log_and_mark(event_name: str, start_iso: str, t0_perf: float):
#     # 1) Always log to CSV (authoritative)
#     append_event(LOG_PATH, event_name, start_iso, t0_perf)
#     # 2) Fire marker requests to all OBS hosts (non-blocking)
#     threads = [Thread(target=add_record_marker_job, args=(t, event_name), daemon=True) for t in TARGETS]
#     for th in threads: th.start()
#     # We don't join here to keep the UI snappy; markers will land asynchronously.

def append_event(path: Path, event_name: str, start_iso: str, t0_perf: float):
    now_utc = datetime.now(timezone.utc)
    elapsed_s = time.perf_counter() - t0_perf
    with open(path, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow([event_name, now_utc.isoformat(), f"{elapsed_s:.3f}"])
    print(f'Logged: {event_name} | {now_utc.isoformat()} | {elapsed_s:.3f}s')

# def add_record_marker_job(tgt, marker_text):
#     cl = obs.ReqClient(host=tgt["host"], port=tgt["port"],
#                        password=tgt["password"], timeout=3)
#     name = tgt.get("name", f'{tgt["host"]}:{tgt["port"]}')
#     try:
#         try:
#             cl.create_record_marker(marker_text, "")
#         except TypeError:
#             cl.create_record_marker(marker_text)
#         # optional: print(f'[{name}] marker added')
#     except Exception as e:
#         print(f'[{name}] Marker error: {e}')

# def add_record_marker_job(tgt, marker_text):
#     """Send a chapter/record marker to OBS using raw send(); falls back across known variants."""
#     cl = obs.ReqClient(host=tgt["host"], port=tgt["port"],
#                        password=tgt["password"], timeout=3)
#     name = tgt.get("name", f'{tgt["host"]}:{tgt["port"]}')
#     try:
#         # 1) Try the common CreateRecordMarker shape
#         try:
#             resp = cl.send("CreateRecordMarker",
#                            {"comment": str(marker_text), "details": ""}, raw=True)
#             if VERBOSE_MARKERS:
#                 print(f'[{name}] Marker ack (CreateRecordMarker): {resp}')
#             return
#         except OBSSDKRequestError as e:
#             # Unknown request or bad params? fall through to the next variant
#             if VERBOSE_MARKERS:
#                 print(f'[{name}] CreateRecordMarker failed ({e.code}); trying CreateRecordChapter…')

#         # 2) Try alternate name/param used by some tools
#         try:
#             resp = cl.send("CreateRecordChapter",
#                            {"chapterName": str(marker_text)}, raw=True)
#             if VERBOSE_MARKERS:
#                 print(f'[{name}] Marker ack (CreateRecordChapter): {resp}')
#             return
#         except OBSSDKRequestError as e2:
#             print(f'[{name}] Marker request not supported (codes: {getattr(e2, "code", "?")}). '
#                   f'CSV log remains authoritative.')

#     except Exception as e:
#         # Network/timeouts or other client issues
#         print(f'[{name}] Marker error: {e}')

def run_keyboard_logger():
    """
    Keyboard logger:
      - EVENT_KEYS map keys -> event names (logged once per press)
      - ESC stops just the event logger (recordings keep running)
      - Q stops OBS recordings and then exits
      - Writes CSV rows with Event Name, Clock Time (UTC ISO), Elapsed Time (s)
    """
    init_csv_new(LOG_PATH)
    start_clock_iso = datetime.now(timezone.utc).isoformat()
    t0_perf = time.perf_counter()

    print("\nEvent logger active. Press keys to log events:")
    for k, name in EVENT_KEYS.items():
        print(f"  [{k}] -> {name}")
    print("Press [ESC] to stop logging only.")
    print("Press [Q]   to stop OBS recording (and logging).\n")

    # 1) Register event hotkeys
    event_hotkeys = []
    for key, event_name in EVENT_KEYS.items():
        h = keyboard.add_hotkey(
            key,
            lambda nm=event_name: append_event(LOG_PATH, nm, start_clock_iso, t0_perf)
            if logging_active.is_set() else None
        )
        event_hotkeys.append(h)

    # 2) Setup flags
    global logging_active
    logging_active = threading.Event()
    logging_active.set()  # start with logging enabled
    stop_and_stop_recording = threading.Event()

    # ESC stops logging (one-way)
    def stop_logging():
        if logging_active.is_set():
            logging_active.clear()
            print("Event logging stopped (recordings still running).")

    hk_esc = keyboard.add_hotkey("esc", stop_logging)

    # Q stops recording and exits
    hk_q = keyboard.add_hotkey("q", stop_and_stop_recording.set)

    # 3) Wait until Q is pressed
    while not stop_and_stop_recording.is_set():
        time.sleep(0.1)

    if logging_active.is_set():
        print("Stopping OBS recordings and logger...")
    else:
        print("Stopping OBS recordings...")

    # Stop all OBS recordings
    threads = [Thread(target=stop_record_job, args=(t,), daemon=True) for t in TARGETS]
    for th in threads:
        th.start()
    for th in threads:
        th.join()

    # 4) Cleanup
    for h in event_hotkeys:
        keyboard.remove_hotkey(h)
    keyboard.remove_hotkey(hk_esc)
    keyboard.remove_hotkey(hk_q)
    print("All done.\n")

def main():
    # Early exit if no OBS instances
    if not TARGETS:
        print("No targets configured.")
        return
    
    # Threads wait for all OBS instances on the network
    barrier = Barrier(len(TARGETS))
    
    # Creates on thread per OBS computer & starts recording
    threads = [Thread(target=start_record_job, args=(t, barrier), daemon=True) for t in TARGETS]
    
    # Start all threads
    for th in threads: th.start()
    for th in threads: th.join()
    
    ## Start keyboard based event logger
    run_keyboard_logger()

main()

[AIR COMPUTER] Recording started.
[GROUND COMPUTER] Recording started.

Event logger active. Press keys to log events:
  [1] -> Stimulus onset
  [2] -> Condition start
  [3] -> Condition end
  [space] -> Default manual marker
Press [ESC] to stop logging only.
Press [Q]   to stop OBS recording (and logging).

Logged: Stimulus onset | 2025-08-28T20:53:03.314634+00:00 | 4.285s
Logged: Condition start | 2025-08-28T20:53:04.470431+00:00 | 5.440s
Logged: Stimulus onset | 2025-08-28T20:53:05.381424+00:00 | 6.351s
Logged: Condition end | 2025-08-28T20:53:06.105118+00:00 | 7.075s
Logged: Condition start | 2025-08-28T20:53:06.527431+00:00 | 7.497s
Logged: Stimulus onset | 2025-08-28T20:53:06.923906+00:00 | 7.893s
Logged: Default manual marker | 2025-08-28T20:53:07.900172+00:00 | 8.870s
Logged: Default manual marker | 2025-08-28T20:53:08.221712+00:00 | 9.191s
Event logging stopped (recordings still running).
Stopping OBS recordings...
[GROUND COMPUTER] Recording stopped.
[AIR COMPUTER] Recording 